In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import re

#optional setting 
pd.set_option('display.max_rows', 150)

In [ ]:
season_episode_list =[]

for season in range(1,4):
    if season != 3:
        for episode in range(1,21):
            ep_num = str(episode)
            if len(ep_num) <= 1:
                ep_num = str(0)+ep_num
            else: 
                ep_num
            se_ep = str(season) + ep_num
            season_episode_list.append(se_ep)
    else:
        for episode in range(1,22):
            ep_num = str(episode)
            if len(ep_num) <= 1:
                ep_num = str(0)+ep_num
            else: 
                ep_num
            se_ep = str(season) + ep_num
            season_episode_list.append(se_ep)

In [ ]:
season_episode_list

In [ ]:
def getTitle(x):
    url_base = 'http://atla.avatarspirit.net/transcripts.php?num='
    season_episode_num = str(x) # in a function make this x
    response = requests.get(url_base + season_episode_num)
    
    #
    soup = BeautifulSoup(response.content, 'lxml')
    content = soup.find(class_='content')
    title = content.find(class_='welcome')
    title = title.get_text()
    title = title.split("-")[-1].strip()
    #print(title)
    return(title)

In [ ]:
df = pd.DataFrame(columns = ['SeasonEpisodeCode','Title'])

In [ ]:
#populate a df with SeasonEpisodeCodes & Titles
title_list = []
for x in season_episode_list:
    title_list.append(getTitle(x))

In [ ]:
#creation of df frame
df['SeasonEpisodeCode'] = season_episode_list
df['Title'] = title_list

df['Season'] = df['SeasonEpisodeCode']
df['EpisodeNum'] = df['SeasonEpisodeCode']

#obtain season#
def seasonNum(x):
    return(int(str(x)[0]))
#apply SeasonNum to df
df['Season'] = df['Season'].apply(seasonNum)

#obtain episode #
def episodeNum(x):
    return(int(str(x)[-2:]))
#apply EpisodeNum to df
df['EpisodeNum'] = df['EpisodeNum'].apply(episodeNum)

In [ ]:
df

In [ ]:
#get episode text for cleaning
def RawScriptPull(x):
    url_base = 'http://atla.avatarspirit.net/transcripts.php?num='
    season_episode_num = str(x) # in a function make this x
    response = requests.get(url_base + season_episode_num)
    
    episode_code = x
    
    soup = BeautifulSoup(response.content, 'lxml')
    content = soup.find(class_='content')
    raw_script = content.find(class_='content')
    
    #extracts all text in italics - which are stage directions
    for linebreak in raw_script.find_all('i'):
        linebreak.extract()
    
    #extracts all for hyperlinks - which we don't need
    for linebreak in raw_script.find_all('a'):
        linebreak.extract()
    
    #extracts all the breaks
    for linebreak in raw_script.find_all('br'):
        linebreak.extract()
    #leaves us only the bolds... which we can seperate items on
    raw_script = str(raw_script)
    return([episode_code, raw_script])
    
    #file = open(str(x) + ".txt","w")
    #file.write(str(script))

In [ ]:
#cleaning up so still keep some italtics with out extracting them needlessly

url_base = 'http://atla.avatarspirit.net/transcripts.php?num='
season_episode_num = str(103) # in a function make this x
response = requests.get(url_base + season_episode_num)

soup = BeautifulSoup(response.content, 'lxml')
content = soup.find(class_='content')
raw_script = content.find(class_='content')

In [ ]:
raw_script

for linebreak in raw_script.find_all('a'):
        linebreak.extract()
        
for linebreak in raw_script.find_all('br'):
    linebreak.extract()

In [ ]:
raw_script = str(raw_script)

In [ ]:
script = raw_script.split('[End Credits]')[0]
script_clean = script.split("Act I</b></u>")[1]

In [ ]:
script_clean

In [ ]:
script_clean = '<i>(Scene opens with a view of the sun rising over a lake. The camera pans left to where Appa and the group are on the lake shore. One of the members is apparently asleep in a Water Tribe sleeping bag on the ground. Appa is standing, grazing slowly, while a small fire casts a plume of smoke. Switch to a close of up of Appa with Aang facing the camera, sitting in Appa\'s "driver\'s seat." Katara is up in the passenger section doing some kind of work.)</i><b>Aang:</b> <i>(while adjusting Appa\'s reins)</i> Wait \'til you see it, Katara. The Air Temple is one of the most beautiful places in the world.<b>Katara:</b> <i>(cautious)</i> Aang, I know you\'re excited, but it\'s been a hundred years since you\'ve been home.<b>Aang:</b> That\'s why I\'m so excited!<b>Katara:</b> It\'s just that a lot can change in all that time.<b>Aang:</b> I know, but I need to see it for myself.<i>(Aang floats down from Appa\'s back and walks over to Sokka, who is snoring peacefully in his sleeping bag on the ground.)</i><b>Aang:</b> Wake up, Sokka! Air Temple here we come!<i>(Sokka awakens, making groggy noises.)</i><b>Sokka:</b> Sleep now... temple later...<i>(He rolls over and begins to snore again. Camera switches back to Aang; his eyes blink accompanied by an audio effect. He is clearly not pleased. A mischievous grin spreads over his face as he gets an idea. Aang picks up a stick as Katara watches from atop Appa\'s back.)</i><b>Aang:</b> Sokka! Wake up! <i>(He begins to run the stick up and down Sokka\'s sleeping bag.)</i> There\'s a prickle snake in your sleeping bag!<b>Sokka:</b> <i>(startled)</i> Aaahhh! Get it off! Get it off! Aaahhh! <i>(Sokka hops around in his sleeping bag until he loses his balance and falls flat on his face. Katara laughs at him. Sokka looks very irritated.)</i><b>Aang:</b> <i>(pokes his head out from behind Sokka.)</i> Great! You\'re awake. Let\'s go.<i>(Scene fades to a view of a Fire Nation naval yard. Tents and buildings line the right side of the screen, ships line the left and what looks like a railroad line runs down the middle. The screen pans left, eventually resting on Zuko\'s damaged ship. It is much smaller than the other Fire Nation ships. The prow of Zuko\'s ship has been opened, its spout lying on the floor of the navy yard. Two figures have walked down the spout and are entering the naval yard itself. Switch to a close up of Zuko and Iroh as they walk into the yard.)</i><b>Zuko:</b> Uncle, I want the repairs made as quickly as possible. I don\'t want to stay too long and risk losing his trail.<b>Iroh:</b> You mean the Avatar?<b>Zuko:</b> <i>(turning on his uncle angrily)</i> Don\'t mention his name on these docks! Once word gets out that he\'s alive every firebender will be out looking for him and I don\'t want anyone getting in the way.<b>Zhao:</b> <i>(from offscreen)</i> Getting in the way of what, Prince Zuko?<i>(Zuko and Iroh turn to face the camera with somewhat surprised expressions. Cut to Zhao as he approaches them, still speaking. Pan out to show the meeting of the three; Zhao\'s hands are clasped behind his back authoritatively.)</i><b>Zuko:</b> <i>(with distaste)</i> Captain Zhao.<b>Zhao:</b> It\'s Commander now. And General Iroh-- <i>(he bows to Iroh)</i> --great hero of our nation.<b>Iroh:</b> <i>Retired</i> general.<b>Zhao:</b> The Fire Lord\'s brother and son are welcome guests any time. What brings you to my harbor?<b>Iroh:</b> Our ship is being repaired. <i>(The screen expands as Iroh gestures at the heavily damaged bow of Zuko\'s ship.)</i><b>Zhao:</b> That\'s quite a bit of damage.<b>Zuko:</b> Yes... you wouldn\'t believe what happened. <i>(He gives his uncle a sideways glance and immediately passes the buck.)</i> Uncle! Tell Commander Zhao what happened.<i>(Iroh\'s eyes go wide and an audio effect is heard as he blinks in response to the burden Zuko has just placed on him.)</i><b>Iroh:</b> Yes, I will do that. It was incredible. <i>(Leans over and whispers to Zuko.)</i> What... did we crash or something?<b>Zuko:</b> <i>(uncomfortable)</i> Uh, yes! Right into an Earth Kingdom ship.<b>Zhao:</b> Really? You must regale me with all the thrilling details. <i>(He smiles and puts his face right up to Zuko\'s in an obvious challenge.)</i> Join me for a drink?<b>Zuko:</b> Sorry, but we have to go.<i>(Zuko turns to leave, but Iroh places a hand on his shoulder and stops him.)</i><b>Iroh:</b> Prince Zuko, show Commander Zhao your respect. <i>(Turning to Zhao.)</i> We would be honored to join you. Do you have any ginseng tea? It\'s my favorite.<i>(Zuko growls in frustration and releases fire from his fists angrily as he turns to follow Zhao and Iroh down the dock.)</i><i>(Switch to view of Appa\'s back, Aang and Katara up front, Sokka in the passenger saddle. The camera pans in on Sokka slowly and then switches to a close up of Sokka\'s stomach, which growls.)</i><b>Sokka:</b> <i>(annoyed)</i> Hey, stomach, be quiet, all right? I\'m trying to find us some food. <i>(He searches the food bag and dumps it out onto his glove. Only a few crumbs come out.)</i><b>Sokka:</b> Hey! Who ate all my blubbered seal jerky?<b>Aang:</b> Oh, that was <i>food</i>? I used it to start the campfire last night. Sorry.<b>Sokka:</b> <i>(incredulous)</i> You WHAT? Awww, no wonder the flames smelled so good.<i>(Pan shot of Appa swooping towards the now much closer mountains, followed by another close up of Aang and the group.)</i><b>Aang:</b> The Batola mountain range! We\'re almost there!<b>Katara:</b> <i>(uneasy)</i> Aang? Before we get to the temple, I want to talk to you about the airbenders.<b>Aang:</b> What about \'em?<b>Katara:</b> <i>(close-up of Katara\'s face.)</i> Well, I just want you to be prepared for what you might see. The Fire Nation is ruthless. They killed my mother and they could have done the same to your people.<b>Aang:</b> <i>(close-up of Aang\'s face, which relaxes into hopeful optimism)</i> Just because no one has <i>seen</i> an airbender doesn\'t mean the Fire Nation killed them all. <i>(Cut to a front view of both, sitting up front.)</i> They probably escaped.<b>Katara:</b> I know it\'s hard to accept.<b>Aang:</b> You don\'t understand, Katara. The only way to get to an airbender temple is on a flying bison, and I doubt the Fire Nation has any flying bison. Right, Appa?<i>(Aang rubs the big animal\'s head. Screen cuts to show a full view of Appa, who grunts an affirmative.)</i><b>Aang:</b> <i>(voice-over)</i> Yip yip! <i>(Appa flies them up higher, turning tightly around a mountain face. Katara and Sokka are straining against the wind sheer the faster they move. Appa flies up and over a rocky outcropping and treetops to reveal the tall spires of the Southern Air Temple behind it.)</i><b>Aang:</b> There it is... the Southern Air Temple.<b>Katara:</b> Aang, it\'s amazing!<b>Aang:</b> <i>(to Appa)</i> We\'re home, buddy. We\'re home.<i>(Scene cuts back to the Fire Nation navy yard. The shot pans right away from the ships, revealing a wooden stockade and many Fire Nation tents behind it. One is particularly large, with a prominent Fire Nation symbol on it. Two guards, flanked by large standing firepots, stand at the ready in front of this tent. The scene cuts again to the interior of the large tent where Commander Zhao stands with his back to the camera, inspecting a large map of the world upon the wall.)</i><b>Zhao:</b> <i>(panning back from him slowly.)</i> And by year\'s end, the Earth Kingdom capital will be under our rule. <i>(Camera cuts to a view of the entire room. Iroh is inspecting a stand of weapons on the left and Zuko sits in one of two chairs in front of Zhao. Zhao turns to Zuko.)</i> <b>Zhao:</b> The Fire Lord will finally claim victory in this war.<b>Zuko:</b> <i>(Cut to Zuko\'s upper body.)</i> If my father thinks the rest of the world will follow him willingly, then he is a fool. <i>(Zhao sits in the chair next to Zuko.)</i><b>Zhao:</b> Two years at sea have done little to temper your tongue. <i>(pause)</i> So, how is your search for the Avatar going?<i>(Iroh tips over the stand of weapons he has been examining in the background. The commotion brings the conversation to a halt. Iroh cringes at the mess he has just made.)</i><b>Iroh:</b> <i>(embarrassed)</i> My fault entirely. <i>(He sheepishly backs o.c. to the left. Zhao watches him move off.)</i><b>Zuko:</b> <i>(cut to Zuko.)</i> We haven\'t found him yet.<b>Zhao:</b> <i>(cut to Zhao.)</i> Did you really expect to? The Avatar died a hundred years ago-- <i>(cut back to Zuko.)</i> --along with the rest of the airbenders. <i>Zuko averts his eyes guiltily. Cut back to Zhao\'s eager face.)</i><b>Zhao:</b> Unless you found some evidence that the Avatar is alive.<b>Zuko:</b> <i>(still looking away)</i> No. Nothing.<b>Zhao:</b> <i>(rising from his chair)</i> Prince Zuko, the Avatar is the only one who can stop the Fire Nation from winning this war. <i>(cut to front view of both.)</i> If you have an <i>ounce</i>-- <i>(he leans his face into Zuko\'s)</i> --of loyalty left, you\'ll tell me what you\'ve found.<b>Zuko:</b> <i>(defiantly)</i> I haven\'t <i>found</i> anything. It\'s like you said. The Avatar probably died a long time ago. Come on, Uncle, we\'re going.<i>(Zuko gets up and tries to exit, but he is blocked by the guards as they cross their spears in front of him. Another guard approaches Zhao to deliver his report.)</i><b>Guard:</b> Commander Zhao, we interrogated the crew as you instructed. They confirmed Prince Zuko had the Avatar in custody, but let him escape.<b>Zhao:</b> Now, remind me... <i>(Cut to Zuko still blocked by the guards. Zhao comes up behind him.)</i><b>Zhao:</b> ... how exactly was your ship damaged? <i>(Zuko looks down in defeat.)</i><i>(Cut to a brief scenic shot of the temple. It is quickly replaced by a long shot of Appa standing on what looks like a landing platform with a path leading up. The screen pans up hundreds of feet winding path curving back and forth across the rock face of the temple\'s mountain. Three little figures can be seen walking up toward the temple. One of these is racing ahead of the others. Cut to a close shot of the three; Aang is the one ahead and he races off screen as Katara and Sokka round the bend.)</i><b>Sokka:</b> So where do I get something to eat? <i>(Cut again to a close up of the two siblings. Sokka has a very cross look on his face and he\'s clutching his stomach to illustrate his hunger.)</i><b>Katara:</b> You\'re lucky enough to be one of the first outsiders to <i>ever</i> visit an airbender temple and all you can think about is food? <i>(Camera cuts to follow them from above and behind.)</i><b>Sokka:</b> I\'m just a simple guy with simple needs.<i>(Cut again to show Aang at the edge of the path where he has stopped to let the others catch up. They do, and he points below them.)</i><b>Aang:</b> So that\'s where my friends and I would play airball!<i>(Aang gestures at a small parapet cut into the rock face below the path. It is populated with a thicket of densely packed sticks of varying heights that have been stuck in the ground. A goal with a backboard occupies either end of the field.)</i><b>Aang:</b> And... over there would be where the bison would sleep... and... <i>(His voice trails off. He sighs.)</i><b>Katara:</b> What\'s wrong?<b>Aang:</b> This place used to be full of monks and lemurs and bison. Now there\'s just a bunch of weeds. <i>(Close up of Aang. He looks sad. Cuts immediately to show Aang from behind as the camera pans back to bring Katara and Sokka into view.)</i> <b>Aang:</b> I can\'t believe how much things have changed.<i>(Sokka and Katara look at each, then change the subject, hurrying up to him.)</i> <b>Sokka:</b> So, uh, this airball game? How do you play? <i>(Aang smiles.)</i><i>(Scene changes to Sokka with a backboard behind him, looking ready for action. Cut to Aang with the opposite backboard behind him. He is bending air around a rapidly spinning ball to keep it floating above his outstretched right hand. Suddenly, Aang starts manipulating the ball like a Harlem Globetrotter. Then, he throws it up in the air over his head. Sokka follows it with his eyes. Aang puts his hands behind his back, closes his eyes and smiles, waiting for the ball to come back down. He cracks one eye open and bends the air at the precise moment to send the ball bouncing through the field of sticks like a pinball in an arcade game. The POV switches to Sokka, who watches the ball ricochet toward him at amazing speed. The ball hits him in the stomach and he is propelled backward through a rotating door in the backboard behind him. He hits the ground nearby.)</i><b>Aang:</b> Hahaha! Aang seven, Sokka zero! <i>(He shows the score with his hands, seven fingers for him and a zero between his touching fingertips for Sokka.)</i><b>Sokka:</b> <i>(getting up painfully)</i> Making him feel better is putting me in a world of hurt.<i>(Sokka sees something that makes him stop and move forward. It is a Fire soldier helmet lying on the ground. Sokka crawls over to it.)</i><b>Sokka:</b> Katara, check this out.<b>Katara:</b> <i>(seeing the helmet, accusingly)</i> Fire Nation.<b>Sokka:</b> We should tell him.<b>Katara:</b> <i>(turning to call to Aang)</i> Aang, there\'s something you need to see.<b>Aang:</b> <i>(approaching happily with the ball)</i> Okay!<i>(Katara looks back and forth between the helmet and the happy boy rapidly approaching, unsure whether to show him the horrible discovery. Making her choice, she waterbends the snow on the nearby bank down onto both the helmet and Sokka.)</i><b>Aang:</b> What is it?<b>Katara:</b> <i>(uncertainly)</i> Uh... just a new waterbending move I learned.<b>Aang:</b> Nice one. But enough practicing. We have a whole temple to see! <i>(He walks o.c. Cut back to Katara and Sokka, who stands and wipes the snow off his shoulders and head.)</i><b>Sokka:</b> You know, you can\'t protect him forever.<u><b>Act II</b></u><i>(The scene has changed to the entrance gate of the Air Temple itself. Aang runs in alone, leaving Sokka and Katara a moment to speak alone.)</i><b>Sokka:</b> Katara, firebenders were here. You can\'t pretend they weren\'t.<b>Katara:</b> <i>(walking away from him)</i> I can for Aang\'s sake. <i>(Sokka comes up behind her and they walk forward together.)</i><b>Katara:</b> If he finds out that the Fire Nation invaded his home, he\'ll be devastated.<b>Aang:</b> <i>(calls from o.c.)</i> Hey guys! <i>(Cut to him motioning toward a statue of an airbender monk.)</i><b>Aang:</b> I want you to meet somebody.<b>Sokka:</b> Who\'s that?<b>Aang:</b> Monk Gyatso, the greatest airbender in the world. He taught me everything I know. <i>(Aang bows to the statue.)</i><i>(Camera closes in on the statue as the scene shifts to a hazy and surreal flashback sequence. A live version of the man whose statue Aang bowed to begins to speak.)</i><b>Gyatso:</b> But the true secret...<i>(View expands to show Gyatso holding a long spatula with a cake at the end of it. He appears to have just pulled out the cake from the mouth of the large oven in the background. He airbends the colorful filling into an attractive pile in the middle of it.)</i><b>Gyatso:</b> ... is in the gooey center!<i>(View expands again to show a preoccupied Aang in the foreground, sitting on the wall of the parapet. He\'s not paying attention.)</i><b>Aang:</b> Hmm...<b>Gyatso</b>: My ancient cake making technique isn\'t the only thing on your mind, is it, Aang?<b>Aang:</b> This whole Avatar thing... maybe the monks made a mistake.<i>(Camera focuses on Gyatso in the background.)</i><b>Gyatso:</b> The only mistake they made was telling you before you turned sixteen. But we can\'t concern ourselves with what <i>was</i>. We must act on what <i>is</i>.<i>(Gyatso gestures ceremoniously to the world around them. The view expands to show the air around the temple full of flying bison and other inhabitants. The beautiful, dreamlike surroundings are full of life.)</i><b>Aang:</b> <i>(o.c.)</i> But Gyatso, how do I know if I\'m ready for this?<b>Gyatso:</b> <i>(switch to view of Gyatso and Aang)</i> Your questions will be answered when you are old enough to enter the Air Temple sanctuary. <i>(Cut to Aang, while Gyatso continues o.c.)</i> Inside you will meet someone who will guide you on your journey.<b>Aang:</b> <i>(excitedly as he jumps up to turn and face Gyatso fully)</i> Who is it?<b>Gyatso:</b> When you are ready he will reveal himself to you. <i>(Aang sighs heavily in frustration.)</i><b>Gyatso:</b> Now, are you going to help me with these cakes, or not?<b>Aang:</b> <i>(smiling)</i> All right.<i>(Aang and Gyatso assume airbending stances, with the four cakes sitting on the wall. They both cock back, creating balls of wind with their airbending skill as Gyatso counts.)</i><b>Gyatso:</b> One... two... three!<i>(Both release their airballs on the count of three, propelling the cakes high into the air. They make a distinctive whizzing noise as they come down. The view switches to show four meditating monks, who are soon each wearing one of the four cakes. Sound effects mark each splattering as the cakes land, and the monks are soon surrounded by winged lemurs who begin feasting on the cakes.)</i><b>Aang</b> and <b>Gyatso:</b> Hahaha!<i>(Aang and Gyatso bow to each other respectfully. Gyatso pats his young student\'s head affectionately.)</i><b>Gyatso:</b> Your aim has improved greatly my young pupil.<i>(The flashback ends, with Aang bowing to the statue of Gyatso, just as he had bowed to the real Gyatso at the end of the flashback.)</i><b>Katara:</b> <i>(coming forward and placing a hand on Aang\'s shoulder.)</i> You must miss him.<b>Aang:</b> Yeah. <i>(moves forward to go into the Temple)</i><b>Katara:</b> Where are you going?<b>Aang:</b> <i>(climbing the steps.)</i> The Air Temple Sanctuary. There\'s someone I\'m ready to meet. <i>(Katara looks at Sokka, who shrugs. View shifts to an overhead shot of the three as they approach the entrance to the Air Temple Sanctuary. It is a huge wooden door that is dominated by an enormous woodcut comprised of three air symbols protruding from its surface. They are arranged in a triangular pattern. The symbols are attached to tubes that end in two horns near the bottom of the woodcut.)</i><b>Katara:</b> But Aang... no one could have survived in there for a hundred years.<b>Aang:</b> It\'s not impossible. I survived in the iceberg for that long.<b>Katara:</b> Good point.<b>Aang:</b> Katara, whoever\'s in there might help me figure out this Avatar thing!<b>Sokka:</b> <i>(Pops out from behind Aang eagerly.)</i> And whoever\'s in there might have a medley of delicious, cured meats!<i>(Sokka rubs his hands together in fevered anticipation of the food that might await him. He rushes forward and runs straight into the door with a \'thunk.\' He strains against the big door to no avail and slides down to the floor in defeat.)</i><b>Sokka:</b> I don\'t suppose you have a key?<b>Aang:</b> The key, Sokka, is airbending.<i>(There\'s a flash of light and the scene cuts to Aang\'s face as he composes himself and draws in his breath. He raises both his arms perpendicular to his body, then suddenly pushes forward with his arms and steps forward with his right leg. He airbends two jets of air, one from each arm, into the horns at the bottom of the woodcut. The air runs through the tubes and one by one flips the air symbols from the blue sides which had been showing to the maroon sides, which had been facing the interior of the temple. As each turns, it flips another mechanism on the outside of the door unlock it. The two leaves of the door open to reveal the dark, cavernous interior of the Air Temple Sanctuary.)</i><b>Aang:</b> <i>(calling inside)</i> Hello? Anyone home? <i>(Cut to a wide shot of Aang walking into the dark room, then back to Katara and Sokka, who follow him. Scene fades to black.)</i><i>(Zhao crosses in front of a seated and clearly unhappy Prince Zuko. A pair of guards stand behind the young prince.)</i><b>Zhao:</b> So, a twelve-year-old boy bested you and your firebenders? <i>(View expands to show Iroh also seated nearby.)</i> You\'re more pathetic than I thought.<b>Zuko:</b> I underestimated him once, but it will not happen again.<i>(Cut to Zhao\'s face with Zuko in the background.)</i><b>Zhao:</b> No, it will not, because you won\'t have a second chance.<b>Zuko:</b> <i>(alarmed)</i> Commander Zhao, I\'ve been hunting the Avatar for two years and I...<i>(Zhao turns on him angrily, flames erupting from his hand as he sweeps it in an arc from left to right.)</i><b>Zhao:</b> And you failed! <i>(Camera pans upward on Zhao\'s face as he towers over Zuko.)</i><b>Zhao:</b> Capturing the Avatar is too important to leave in a teenager\'s hands. He\'s mine now. <i>(Zuko launches himself at Zhao in frustration and anger, but he is restrained by the two guards standing behind his chair.)</i><b>Zhao:</b> <i>(to the guards as he turns to leave.)</i> Keep them here.<i>(Zuko in a further act of frustration kicks over a small table that had been sitting near Iroh. It breaks into pieces. Iroh watches the scene calmly.)</i><b>Iroh:</b> More tea please?<i>(Cut to Aang, Sokka, and Katara as they walk into the sanctuary. The camera pans right to reveal many statues arrayed in a pattern. The pattern is tied to a swirl pattern on the floor--a bluish version of the Yellow Brick Road--with the statues arrayed along it. The three walk among the statues, heading to the center of the room.)</i><b>Sokka:</b> Statues?! That\'s it? Where\'s the meat? <i>(Scene cuts to Aang and Katara.)</i><b>Katara:</b> Who are all these people?<b>Aang:</b> <i>(uncertain)</i> I\'m not sure, but it feels like I know them somehow. Look! <i>(He points to a statue.)</i> That one\'s an airbender!<b>Katara:</b> <i>(pointing)</i> And this one\'s a waterbender. They\'re lined up in a pattern. Air, water, earth and fire.<b>Aang:</b> That\'s the Avatar cycle.<b>Katara:</b> Of course! They\'re Avatars. All these people are your past lives, Aang.<b>Aang:</b> Wow! There\'s so many!<i>(The camera stays behind and shifts upward as Aang moves off screen, showing the many tiers of statues to illustrate just how many lives Aang has had before him.)</i><b>Sokka:</b> <i>(skeptical)</i> Past lives? Katara, you really believe in that stuff?<b>Katara:</b> It\'s true. When the Avatar dies he\'s reincarnated into the next nation in the cycle.<i>(Aang has stopped in front of a statue of a firebender Avatar. The POV flips back and forth between Aang and the statue. A light passes over its eyes, marking its importance. Katara appears behind Aang and shakes him by the shoulders).</i><b>Katara:</b> Aang, snap out of it!<b>Aang:</b> <i>(dazed)</i> Huh?<b>Katara:</b> Who is that?<b>Aang:</b> That\'s Avatar Roku, the Avatar before me.<b>Sokka:</b> You were a firebender? No wonder I didn\'t trust you when we first met.<b>Katara:</b> There\'s no writing. How do you know his name?<b>Aang:</b> I\'m not sure... I just know it somehow.<b>Sokka:</b> <i>(growls in frustration)</i> You just couldn\'t get any weirder!<i>(The three sense the presence of another being and turn to look at the entrance. A long-eared shadow advances toward them. The screen pans back up, but the three are nowhere to be seen. When the view comes to rest again, we can see them huddled behind two of the statues, Aang and Katara behind one on the left of the screen, Sokka behind one on the right. The shadow advances between the two statues. Screen cuts to a profile view where we can see their frightened faces.)</i><b>Sokka:</b> <i>(whispering)</i> Firebender. Nobody make a sound.<b>Katara:</b> <i>(exasperated)</i> You\'re making a sound!<b>Aang</b> and <b>Sokka:</b> Shhhh!<i>(The shadow advances. Screen cuts to black.)</i><u><b>Act III</b></u><i>(Fade in on the still advancing shadow.)</i><b>Sokka:</b> <i>(whispering, ready with his weapon)</i> That firebender won\'t know what hit \'em.<i>(The long eared shadow is now right on top of them. The view pans up to reveal the black outline of a small animal in the doorway who is simply casting a lengthy shadow due to the angle of the sun outside. Camera cuts to behind the animal as Sokka jumps out from the statue, weapon at the ready. The other two also come around to look. Sound effects are heard as everyone\'s eyes blink, each registering the diminutive stature of the adorable intruder. The view switches to reveal a winged lemur looking at them with wide eyes. His long ears flop down on his back as he sees the people staring at him. He blinks his eyes to the same sounds effects. The camera gives a close-up of Aang.)</i><b>Aang:</b> Lemur!<b>Sokka:</b> <i>(cut to Sokka, drooling)</i> Dinner...<b>Aang:</b> Don\'t listen to him! You\'re going to be my new pet.<b>Sokka:</b> Not if I get him first!<i>(Both lunge at the small animal. The scene shifts to a joint POV of both Aang and Sokka, both with their bodies off screen, but their arms visibly outstretched towards the lemur as they rapidly close in on him. He bristles like a cat and dashes away from them, screaming. View changes to see them both running after the lemur who is now bolting out of the temple.)</i><b>Aang:</b> <i>(calling)</i> Wait! Come back!<i>(Cuts to long shot of the entrance hallway. The lemur rockets out of the screen.)</i><b>Sokka:</b> I wanna eat you!<i>(Aang and Sokka run back down the hallway from the Temple Sanctuary, chasing after the lemur. Both take turns leading the other. Sokka takes a swipe at Aang\'s legs with his weapon, but Aang, uses his airbending skills in a Matrix-style move to run along the wall beside Sokka and pass him, laughing as he does. Once far enough ahead, Aang stops, turns, and launches an airball spinning back toward Sokka. It hits him in the stomach and knocks him off his feet, like a bowling ball hitting one lone pin.)</i><b>Sokka:</b> Oof! <i>(He falls flat on his face, his hood falling over his head.)</i>(Scene changes to where the hallway ends at a balcony, looking back into the temple. The lemur jumps up onto the guardrail of the balcony, looks back as Aang approaches, and then jumps off the guardrail almost into the camera. Aang jumps off after him, with the view expanding as he jumps off to provide a clear look at the long fall beneath him. Aang laughs as he falls, bouncing off the rocks beneath him as he continues to chase the lemur. The scene cuts to Sokka, who finally reaches the balcony, leaning over to watch Aang fall.)<b>Sokka:</b> Hey! No Fair!<i>(View switches to a close up of Aang, eyes watering from the tremendous speed his body is falling at and smiling widely.)</i><i>(Zhao approaches the exterior of the large tent, opens the flaps and enters. Two guards standing ready inside the entrance draw their spears back.)</i><b>Zhao:</b> My search party is ready. <i>(Cut to Iroh and Zuko, sitting in chairs facing each other.)</i><b>Zhao:</b> Once I\'m out to sea, my guards will escort you back to your ship and you\'ll be free to go.<b>Zuko:</b> Why? Are you worried I\'m going to try and stop you?<b>Zhao:</b> <i>(laughing)</i> You? Stop me? Impossible.<b>Zuko:</b> <i>(Zuko stands in defiance.)</i> Don\'t underestimate me, Zhao. I <i>will</i> capture the Avatar <i>before</i> you. <b>Iroh:</b> <i>(standing also)</i> Prince Zuko, that\'s enough!<b>Zhao:</b> You can\'t compete with me. I have hundreds of warships under my command, and you... you\'re just a banished prince. No home. No allies. Your own father doesn\'t even want you.<b>Zuko:</b> You\'re wrong. Once I deliver the Avatar to my father he will welcome me home with honor and restore my rightful place on the throne.<b>Zhao:</b> <i>(cut to sliding right pan of Zhao\'s face)</i> If your father really wanted you home, he\'d have let you return by now, Avatar or no Avatar, but in his eyes you are a failure and a disgrace to the Fire Nation.<b>Zuko:</b> That\'s not true.<b>Zhao:</b> You have the scar to prove it.<b>Zuko:</b> <i>(With a cry of indignation, launches himself to his feet, bringing his face within inches of Zhao\'s.)</i> Maybe you\'d like one to match!<b>Zhao:</b> Is that a challenge?<b>Zuko:</b> An <i>agni kai</i>. At sunset.<b>Zhao:</b> Very well. It\'s a shame your father won\'t be here to watch me humiliate you. I guess your uncle will do.<i>(Zhao turns and walks back out of the tent. The camera cuts to show a close-up of the left side of Zuko\'s face. Iroh visible in the background as the camera pans quickly to the right.)</i><b>Iroh:</b> Prince Zuko, have you forgotten what happened last time you dueled a master?<b>Zuko:</b> I will never forget. <i>(The scene ends with only the right half of Zuko\'s face, the side with the scar, visible on the screen.)</i><i>(A quick shot of the exterior of the temple atop the mountain is replaced by a view of a stone clearing that has an ascending stone staircase in the background. The lemur that Aang has been chasing lands on the ground. Aang pounces him and misses. The view pans left as the lemur runs through a curtain made of decaying tent material or drapery.)</i><b>Aang:</b> Hey! Come back! <i>(Aang follows the lemur to the other side of the curtain.)</i> <b>Aang:</b> Come on out, little lemur. That hungry guy won\'t bother you anymore.<i>(Aang approaches another drape, parts it, and walks through. He draws a sharp breath, startled at what he sees. Cut to show heaps of firebender uniforms and skeletons covering the floor of a decaying building.)</i><b>Aang:</b> <i>(surprised)</i> Firebenders? They were here?<i>(At the back of the grotto lies the skeletal remains of an airbender monk, bathed in sunlight coming from above, who had apparently fallen in combat fighting against great odds. The camera zooms in on the airbender\'s necklace as a means of identification.)</i><b>Aang:</b> Gyatso... <i>(He falls to his knees, devastated.)</i><b>Sokka:</b> <i>(pulling back the curtain)</i> Hey Aang, you find my dinner yet? <i>(He sees Aang, head in his hands, crying.)</i> <b>Sokka:</b> Aang, I wasn\'t really going to eat the lemur, okay? <i>(He sees the skeleton and does a doubletake.)</i> Oh, man... come on, Aang, everything will be all right. Let\'s get out of here.<i>(Sokka puts his hand on Aang\'s shoulder as the arrow on Aang\'s head begins to glow incandescent blue. The camera shifts to Aang\'s face, his eyes are glowing brightly in an angry expression as he lifts his head. Sokka gasps as he looks on in alarm.)(Scene shifts to Katara walking among the statues in the Temple Sanctuary. She stops in front of Roku, whose eyes light up with the same incandescent blue light as Aang\'s. The eyes of all the other statues light up in order around the room. The camera pans up so the audience can see all the statues\' eyes light up in succession.)</i><b>Katara:</b> <i>(worried)</i> Aang!<i>(Katara races from the room. Scene shifts to earth, water, and fire temples scattered across the globe, where lights flash in response to the awakening of Aang\'s avatar spirit. In the fire temple, an old fire sage leans out to say something to another sage outside of the room where the signal is emanating from.)</i><b>Old Sage:</b> Send word to the Fire Lord immediately. The Avatar has returned! <i>(Recipient fire sage\'s face becomes frightened.)</i><i>(Scene shifts back to an exterior view of the dilapidated building at the base of the temple where Aang mourns Gyatso\'s body. The view shifts again to Aang\'s feet, where a whirlwind begins to form, air swirling fast around the bones lying on the ground. The camera pans up to Aang, crouched, his hands balled into fists and his eyes and arrow glowing, the wind picking up terrible speed around him.)</i><b>Sokka:</b> Aang! Come on, snap out of it!<i>(A light blue sphere of energy surrounds Aang and begins to expand, knocking Sokka back out of the building.)</i><b>Sokka:</b> Aaahhh!<i>(The energy sphere expands to the point where it blows most of the building apart, sending smoke and debris high into the air, which the camera follows. Sokka lands outside the building. The camera switches back to Aang, now partially obscured by the energy sphere and the maelstrom. Katara joins Sokka behind some rubble, both shielding their faces from the wind.)</i><b>Katara:</b> What happened?<b>Sokka:</b> He found out firebenders killed Gyatso.<b>Katara:</b> Oh no, it\'s his avatar spirit! He must have triggered it! I\'m gonna try and calm him down.<b>Sokka:</b> <i>(Hanging onto the rubble, trying not to get blown away.)</i> Well, do it before he blows us off the mountain! <i>(Katara slowly approaches Aang, struggling against the wind, as Aang and his energy sphere slowly rise into the air.)(Cut to an exterior view of a Fire Nation arena, a simple four-walled structure with watch fires on the four towers at each corner. There is one large open gate. The sunset is a glorious mixture of Fire Nation hues: red, orange, and yellow. The view shifts to inside the arena where Zhao and Zuko kneel, preparing for battle, Zhao with four of his men in attendance, Zuko with his uncle.)</i><b>Iroh:</b> Remember your firebending basics, Prince Zuko. They are your greatest weapons.<b>Zuko:</b> <i>(standing up)</i> I refuse to let him win. <i>(His shoulder wrap falls to the ground ceremoniously. View shifts to Zhao. He stands and turns, his shoulder wrap falling to the ground, as well.)</i><b>Zhao:</b> This will be over quickly. <i>(Atop the gate a gong sounds. Both men face each other and assume firebending stances. The screen splits: the upper pane is a close up of Zhao\'s face, the lower a close up of Zuko\'s. Prince Zuko fires the first shot which passes harmlessly to Zhao\'s left. He fires again; this time it passes without effect to Zhao\'s right. Zuko fires several more, the last of which Zhao blocks, satisfaction evident on his face. Frustrated and losing control of his breath, Zuko moves towards his opponent, unleashing more fire from both his hands and feet. Zhao dodges or blocks them all. Zhao then crouches forward and shoots flame at a point on the ground close in front of him. Camera switches to Iroh, watching anxiously.)</i><b>Iroh:</b> Basics, Zuko! Break his root!<i>(Zhao fires many volleys of flame, alternating between his fists. Zuko blocks each, but is slowly forced back. On the last volley Zhao uses both hands, knocking Zuko over and sending him skidding backwards in the dirt. Zhao takes a flying jump at him. Zhao lands as Zuko tries to get up, but he isn\'t fast enough. Zhao fires right at him. Zuko rolls out of the way just in time, and as he is getting up sweeps Zhao\'s feet out from under him. The move holds such importance that it is shown three times from slightly different camera angles at reduced speed.)(Zuko lands on his feet. Camera switches to a close up of Zuko\'s face, where a slight smile appears. Camera switches to a close up of Zuko\'s feet. Zuko is advancing towards Zhao, using his feet to produce small waves of flame that rush toward his opponent. Zhao is caught off balance and wobbles slowly backward. Camera switches to Iroh, fists clenched in an expression of hope, a smile beginning to spread over his face. Switch back to Zuko, who finally lays Zhao out flat on the ground with a blast of fire. Zuko rushes up to him, prepared to deal the final blow. A close up of Zuko\'s face emphasizes the decision he faces. Cut to Zhao.)</i><b>Zhao:</b> Do it! <i>(Zuko releases a blast that shoots off screen. He stands erect, dropping his fighting stance. View changes to a smoking hole in the ground, and then the screen expands to show Zhao, clearly very much unharmed, next to the hole.)</i><b>Zhao:</b> That\'s it? Your father raised a coward.<b>Zuko:</b> Next time you get in my way, I promise I won\'t hold back.<i>(Zuko turns his back and begins to walk away. Zhao gets up and, with a howl of anger, unleashes a whip of flame at Zuko. The POV changes to the flame itself as it rushes toward Zuko\'s back. The view switches back to Zhao and his outstretched foot, wreathed in the fire it has just released. The flame is extinguished, however, as a hand closes over the foot. The camera changes to a side view, where Iroh has moved between them to stop the dishonorable attack. They remain frozen for a moment before Iroh hurls Zhao back into the dirt with little effort. Zuko rushes to avenge the insult, but is stopped by his uncle.)</i><b>Iroh:</b> No, Prince Zuko. Do not taint your victory. <i>(Iroh turns to face Zhao, a look of condescension on his face.)</i><b>Iroh:</b> So this is how the great Commander Zhao acts in defeat. <i>(Camera switches to Zhao and Iroh continues o.c.)</i> Disgraceful. <i>(Camera returns to Iroh and Zuko.)</i><b>Iroh:</b> Even in exile my nephew is more honorable than you. <i>(Zuko looks at his uncle in surprise at this comment.)</i> Thanks again for the tea. It was delicious.<i>(Iroh moves to leave and Zuko follows close behind. With subdued anger, Zhao watches them exit the gate of the arena. Outside, Zuko and Iroh talk.)</i><b>Zuko:</b> <i>(quietly)</i> Did you really mean that, Uncle?<b>Iroh:</b> <i>(slyly)</i> Of course. I told you ginseng tea is my favorite.<i>(Switch to long shot view of them leaving the arena and walking back to the ships in the harbor.) (Cut to Aang, still suspended in mid-air inside his ball of raw energy. The contained storm continues to rage as Katara and Sokka cling desperately to the rocks at the bottom of the screen.)</i><b>Katara:</b> <i>(shouting calmly)</i> Aang, I know you\'re upset... <i>(Camera switches to full view of Katara clinging to a rock, Sokka visible behind her.)</i><b>Katara:</b> ... and I know how hard it is to lose the people you love. I went through the same thing when I lost my mom. Monk Gyatso and the other airbenders may be gone, but you still have a family. Sokka and I, we\'re your family now.<i>(The view switches back to Aang towards the end of Katara\'s statement, and he begins to descend when she is finished. His feet gently alight on the ground. The wind dies away. Switch to Aang, his eyes and arrow still glowing. Katara and Sokka come up on either side of him.)</i><b>Sokka:</b> <i>(gently)</i> Katara and I aren\'t going to let anything happen to you. Promise.<i>(Katara takes one of Aang\'s hands in her own. The glow instantly fades from his eyes and arrow markings. Grief-stricken and exhausted, he collapses into Katara\'s arms and she holds him around his shoulders as they kneel on the ground.)</i><b>Aang:</b> <i>(tired)</i> I\'m sorry.<b>Katara:</b> It\'s okay. It wasn\'t your fault.<b>Aang:</b> <i>(sadly)</i> But you were right. And if firebenders found this temple that means they found the other ones, too. I really am the last airbender. <i>(Katara holds him tighter and Sokka puts a hand on Aang\'s shoulder. Scene cuts back to the Temple Sanctuary, where Aang stands once again in front of the statue of Avatar Roku. Katara comes up behind him.)</i><b>Katara:</b> Everything\'s packed. You ready to go?<b>Aang:</b> <i>(still looking at the statue)</i> How is Roku supposed to help me if I can\'t talk to him?<b>Katara:</b> Maybe you\'ll find a way.<i>(They both turn around to see a familiar furry creature in the doorway of the temple. The lemur has returned. View switches to behind the lemur, who has Aang and Katara in front of him and Sokka to his right. He runs to Sokka and deposits a bunch of fruits and nuts at his feet. Sokka begins to eat hungrily as the lemur dashes away. Camera cuts back to Aang and Katara, watching with smiles on their faces.)</i><b>Aang:</b> Looks like you made a new friend, Sokka.<b>Sokka:</b> <i>(mouth full)</i> Can\'t talk. Must eat.<b>Aang:</b> <i>(as the lemur scurries up his chest to perch on his head.)</i> Hey little guy.<i>(Scene switches to outside of the Temple. Screen pans down to show Appa, Aang, and the lemur looking out at the abandoned temple.)</i><b>Aang:</b> You, me, and Appa. <i>(View switches to frontal shot of the three.)</i> We\'re all that\'s left of this place. We have to stick together. Katara, Sokka... <i>(Camera switches to Katara and Sokka, the latter\'s mouth still full, a fruit in his hand.)</i><b>Aang:</b> ... say hello to the newest member of our family. <i>(Aang approaches them both, the lemur on his arm.)</i><b>Katara:</b> What are you going to name him?<b>Aang:</b> <i>(surprised as the lemur jumps off camera and returns a second later with a fruit.)</i> Momo.<i>(Screen expands to show all three, with Sokka poised to bite a fruit that is no longer in his hand. Aang and Katara begin to laugh. Switch to a twilight view of the temple. View shifts to Aang watching the temple recede in the distance as Appa flies them away. Aang looks back with sadness as the clouds finally obscure his childhood home from view.)</i>'

In [ ]:
script_clean

In [ ]:
script_clean = "<i>(Scene opens with a view of the sun rising over a lake. The camera pans left to where Appa and the group are on the lake shore. One of the members is apparently asleep in a Water Tribe sleeping bag on the ground. Appa is standing, grazing slowly, while a small fire casts a plume of smoke. Switch to a close of up of Appa with Aang facing the camera, sitting in Appa\'s "driver\'s seat." Katara is up in the passenger section doing some kind of work.)</i><b>Aang:</b> <i>(while adjusting Appa\'s reins)</i> Wait \'til you see it, Katara. The Air Temple is one of the most beautiful places in the world.<b>Katara:</b> <i>(cautious)</i> Aang, I know you\'re excited, but it\'s been a hundred years since you\'ve been home.<b>Aang:</b> That\'s why I\'m so excited!<b>Katara:</b> It\'s just that a lot can change in all that time.<b>Aang:</b> I know, but I need to see it for myself.<i>(Aang floats down from Appa\'s back and walks over to Sokka, who is snoring peacefully in his sleeping bag on the ground.)</i><b>Aang:</b> Wake up, Sokka! Air Temple here we come!<i>(Sokka awakens, making groggy noises.)</i><b>Sokka:</b> Sleep now... temple later...<i>(He rolls over and begins to snore again. Camera switches back to Aang; his eyes blink accompanied by an audio effect. He is clearly not pleased. A mischievous grin spreads over his face as he gets an idea. Aang picks up a stick as Katara watches from atop Appa\'s back.)</i><b>Aang:</b> Sokka! Wake up! <i>(He begins to run the stick up and down Sokka\'s sleeping bag.)</i> There\'s a prickle snake in your sleeping bag!<b>Sokka:</b> <i>(startled)</i> Aaahhh! Get it off! Get it off! Aaahhh! <i>(Sokka hops around in his sleeping bag until he loses his balance and falls flat on his face. Katara laughs at him. Sokka looks very irritated.)</i><b>Aang:</b> <i>(pokes his head out from behind Sokka.)</i> Great! You\'re awake. Let\'s go.<i>(Scene fades to a view of a Fire Nation naval yard. Tents and buildings line the right side of the screen, ships line the left and what looks like a railroad line runs down the middle. The screen pans left, eventually resting on Zuko\'s damaged ship. It is much smaller than the other Fire Nation ships. The prow of Zuko\'s ship has been opened, its spout lying on the floor of the navy yard. Two figures have walked down the spout and are entering the naval yard itself. Switch to a close up of Zuko and Iroh as they walk into the yard.)</i><b>Zuko:</b> Uncle, I want the repairs made as quickly as possible. I don\'t want to stay too long and risk losing his trail.<b>Iroh:</b> You mean the Avatar?<b>Zuko:</b> <i>(turning on his uncle angrily)</i> Don\'t mention his name on these docks! Once word gets out that he\'s alive every firebender will be out looking for him and I don\'t want anyone getting in the way.<b>Zhao:</b> <i>(from offscreen)</i> Getting in the way of what, Prince Zuko?<i>(Zuko and Iroh turn to face the camera with somewhat surprised expressions. Cut to Zhao as he approaches them, still speaking. Pan out to show the meeting of the three; Zhao\'s hands are clasped behind his back authoritatively.)</i><b>Zuko:</b> <i>(with distaste)</i> Captain Zhao.<b>Zhao:</b> It\'s Commander now. And General Iroh-- <i>(he bows to Iroh)</i> --great hero of our nation.<b>Iroh:</b> <i>Retired</i> general.<b>Zhao:</b> The Fire Lord\'s brother and son are welcome guests any time. What brings you to my harbor?<b>Iroh:</b> Our ship is being repaired. <i>(The screen expands as Iroh gestures at the heavily damaged bow of Zuko\'s ship.)</i><b>Zhao:</b> That\'s quite a bit of damage.<b>Zuko:</b> Yes... you wouldn\'t believe what happened. <i>(He gives his uncle a sideways glance and immediately passes the buck.)</i> Uncle! Tell Commander Zhao what happened.<i>(Iroh\'s eyes go wide and an audio effect is heard as he blinks in response to the burden Zuko has just placed on him.)</i><b>Iroh:</b> Yes, I will do that. It was incredible. <i>(Leans over and whispers to Zuko.)</i> What... did we crash or something?<b>Zuko:</b> <i>(uncomfortable)</i> Uh, yes! Right into an Earth Kingdom ship.<b>Zhao:</b> Really? You must regale me with all the thrilling details. <i>(He smiles and puts his face right up to Zuko\'s in an obvious challenge.)</i> Join me for a drink?<b>Zuko:</b> Sorry, but we have to go.<i>(Zuko turns to leave, but Iroh places a hand on his shoulder and stops him.)</i><b>Iroh:</b> Prince Zuko, show Commander Zhao your respect. <i>(Turning to Zhao.)</i> We would be honored to join you. Do you have any ginseng tea? It\'s my favorite.<i>(Zuko growls in frustration and releases fire from his fists angrily as he turns to follow Zhao and Iroh down the dock.)</i><i>(Switch to view of Appa\'s back, Aang and Katara up front, Sokka in the passenger saddle. The camera pans in on Sokka slowly and then switches to a close up of Sokka\'s stomach, which growls.)</i><b>Sokka:</b> <i>(annoyed)</i> Hey, stomach, be quiet, all right? I\'m trying to find us some food. <i>(He searches the food bag and dumps it out onto his glove. Only a few crumbs come out.)</i><b>Sokka:</b> Hey! Who ate all my blubbered seal jerky?<b>Aang:</b> Oh, that was <i>food</i>? I used it to start the campfire last night. Sorry.<b>Sokka:</b> <i>(incredulous)</i> You WHAT? Awww, no wonder the flames smelled so good.<i>(Pan shot of Appa swooping towards the now much closer mountains, followed by another close up of Aang and the group.)</i><b>Aang:</b> The Batola mountain range!"

In [ ]:
counter = script_clean.count('(')
i = 0
print(counter)

In [ ]:
while i <= 10:
    start_index = script_clean.index('(')
    end_index = script_clean.index(')')
    script_clean = script_clean[0:start_index:]+script_clean[end_index+1::]
    #print(script_clean)
    print(i)
    i = i +1
    print()

In [ ]:
script_clean

In [ ]:
script_clean

In [ ]:
print(script_clean)

In [ ]:
start_index = script_clean.index('(')
end_index = script_clean.index(')')
print(start_index)
print(end_index)

In [ ]:
if len(script_clean)>end_index:
    script_clean = script_clean[0:start_index:]+script_clean[end_index+1::]

In [ ]:
script_clean

In [ ]:
start_index = script_clean.index('(')
end_index = script_clean.index(')')
print(start_index)
print(end_index)
print(len(script_clean))

In [ ]:
print(script_clean.count("("))
print(script_clean.count(")"))

In [ ]:
#end_index = script_clean.index(')')
#print(end_index)
i = 0
while i < 10:
    script_clean = script_clean[0:start_index:]+script_clean[end_index+1::]
    print(script_clean)
    print(i)
    print()
    i = i +1

In [ ]:
script_clean

In [ ]:
script_clean

In [ ]:
script_clean.count('(')

In [ ]:
script_clean.count(')')

In [ ]:
script_clean

In [ ]:
i = 0
while i < 154:
    start_index = script_clean.index('(')
    end_index = script_clean.index(')')
    print(end_index)
    if len(script_clean)>end_index:
            script_clean = script_clean[0:start_index:]+script_clean[end_index+1::]
    i= i+1

In [ ]:
if ("(" in script_clean) and (")" in script_clean):
    while ("(" in script_clean) and (")" in script_clean):
        start_index = script_clean.index('(')
        end_index = script_clean.index(')')
        print(end_index)
        if len(script_clean)>end_index:
            script_clean = script_clean[0:start_index:]+script_clean[end_index+1::]

In [ ]:
print(script_clean.count("("))
print(script_clean.count(")"))

In [ ]:
counter = min(script_clean.count("("),script_clean.count(")"))

In [ ]:
counter

In [ ]:
test = '<i>(Scene opens with a view of the sun rising over a lake. The camera pans left to where Appa and the group are on the lake shore. One of the members is apparently asleep in a Water Tribe sleeping bag on the ground. Appa is standing, grazing slowly, while a small fire casts a plume of smoke. Switch to a close of up of Appa with Aang facing the camera, sitting in Appa\'s "driver\'s seat." Katara is up in the passenger section doing some kind of work.)</i><b>Aang:</b> <i>(while adjusting Appa\'s reins)</i> Wait \'til you see it, Katara. The Air Temple is one of the most beautiful places in the world.'

In [ ]:
counter = test.count('(')

In [ ]:
#think this will work
i = 0
while i < counter:
    start_index = test.index('(')
    end_index = test.index(')')
    test = test[0:start_index:]+test[end_index+1::]
    print(test)
    print(i)
    i = i +1
    print()

In [ ]:
start_index = test.index('(')
end_index = test.index(')')
test = test[0:start_index:]+test[end_index+1::]
print(test)

In [ ]:
start_index = test.index('(')
end_index = test.index(')')
test = test[0:start_index:]+test[end_index+1::]
print(test)

In [ ]:
test

In [ ]:
while ("(" in test) and (")" in test):
    start_index = test.index('(')
    end_index = test.index(')')
    if len(test)>end_index:
        test = test[0:start_index:]+test[end_index+1::]

In [ ]:
test

In [ ]:
start_index = test.index('(')
end_index = test.index(')')

In [ ]:
print(start_index)
print(end_index)

In [ ]:
if len(test)>end_index:
    test = test[0:start_index:]+test[end_index+1::]

In [ ]:
test

In [ ]:
start_index = test.index('(')
end_index = test.index(')')

In [ ]:
print(start_index)
print(end_index)

In [ ]:
if len(test)>end_index:
    test = test[0:start_index:]+test[end_index+1::]

In [ ]:
test

In [ ]:
test = test.del(3:451)

In [ ]:
counter = min(test.count("("),test.count(")"))

In [ ]:
counter

In [ ]:
remove = test.partition("(")[2].partition(")")[0]

In [ ]:
remove

In [ ]:
test = test.replace(remove,"")

In [ ]:
test

In [ ]:
remove = test.partition("(")[2].partition(")")[0]

In [ ]:
remove

In [ ]:
test

In [ ]:
remove = test.partition("(")[2].partition(")")[0]

In [ ]:
test = test.replace(remove,"")

In [ ]:
test

In [ ]:
script_clean

In [ ]:
remove = script_clean.partition("(")[2].partition(")")[0]

In [ ]:
new_script_clean = script_clean.replace(remove,"")

In [ ]:
new_script_clean

In [ ]:
"(" in new_script_clean

In [ ]:
while "(" in new_script_clean:
    remove = script_clean.partition("(")[2].partition(")")[0]
    new_script_clean = script_clean.replace(remove,"")

In [ ]:
new_script_clean

In [ ]:
print(new_script_clean.count("("))
print(new_script_clean.count(")"))

In [ ]:
#creating a function to clean the script
#then puts it into a data frame
#and counts the number of words each character says

def scriptClean(script_info):
    #episode_code = locals()
    
    episode_code = script_info[0]
    raw_script = script_info[1]
    
    file_clean = re.sub(r'\([^()]*\)', '', raw_script) #need to better understand how reGex Works
    
    #removes data after the Ending Credits which is extra
    script = file_clean.split('[End Credits]')[0]

    #removes the data before Act I, which is also extra
    #unfortunetly, website uses 2 different conventions, so I split depending on which one it uses
    
    if "Act I</b></u>" in script:
        script_clean = script.split("Act I</b></u>")[1]
        #bold_split = script_clean.split('<b>')
    elif "Act I</u></b>" in script:
        script_clean = script.split("Act I</u></b>")[1]
        #bold_split = script_clean.split('<b>')
    
    #needed for episodes 212
    elif "Act I </u></b>" in script:
        script_clean = script.split("Act I </u></b>")[1]
        
        
    #return(script_clean)
    bold_split = script_clean.split('<b>')
    
    #creates a list of lists with both character + relevant dialogue
    master_list = []
    for x in range(len(bold_split)):
        output = bold_split[x].split(':')
        master_list.append(output)
    
    #collects all the characters who speak
    character_list = []

    for x in range(len(master_list)):
        holder = master_list[x]
        if len(holder)>1:
            test = holder[0].strip()
            character_list.append(test.strip('</b>')) # remove the bold hyperlink to cleanup the data
    #collects what all the characters say
    dialogue_list = []

    for x in range(len(master_list)):
        holder = master_list[x]
        if len(holder)>1:
            test = holder[0].strip()
            dialogue_list.append(holder[1].strip('</b>')) # remove the bold hyperlink to cleanup the data 
    
    
    #create a list of episode numbers to add to df (feels inefficient and should be a better way to do this)
    episode_list = []
    
    for x in range(len(character_list)):
       episode_list.append(episode_code)
    
    dfScript = pd.DataFrame()
    
    #dfScript['episode_code'] = episode_list
    dfScript['episode_code'] = episode_list
    
    dfScript['character_list'] = character_list
    dfScript['dialogue_list'] = dialogue_list
    
    count = dfScript['dialogue_list'].str.split().str.len()
    count = list(count)
    dfScript['word_count']=count

    return(dfScript)

In [ ]:
def scriptCleanTalesofBaSingSe(script_info):
    episode_code = script_info[0]
    raw_script = script_info[1]
    
    file_clean = re.sub(r'\([^()]*\)', '', raw_script) #references RawScriptPull so it can get cleaned 
    
    script = file_clean.split('[End Credits]')[0]
    
    #removes last story of Be Sing Se
    #as 'The Tale of Momo' doesn't have any dialogue in it
    script = script.split('<b><u>The Tale of Momo</u>')[0] 
    
    #takes individual stories and combines them into a single variable
    
    #Toph_Katara
    Toph_Katara = script.split('Written by: Joann Estoesta and Lisa Wahlander')[1]
    Toph_Katara = Toph_Katara.split('<b><u>The Tale of Iroh</u></b>')[0]
    
    #Iroh
    
    Iroh = script.split('<b><u>The Tale of Iroh</u></b>Written by: Andrew Huebner')[1]
    Iroh = Iroh.split('<b><u>The Tale of Aang')[0]
    
    #Aang
    
    Aang = script.split('<b><u>The Tale of Aang</u></b>Written by: Gary Scheppke')[1]
    Aang = Aang.split('<b><u>The Tale of Sokka</u></b>Written by: Lauren Macmullan')[0]
    
    #Sokka
    
    Sokka = script.split('<b><u>The Tale of Sokka</u></b>Written by: Lauren Macmullan')[1]
    Sokka = Sokka.split('<b><u>The Tale of Zuko</u></b>Written by: Andrew Huebne')[0]
    
    #Zuko
    #as last bit, don't need any code to cut it off
    Zuko = script.split('<b><u>The Tale of Zuko</u></b>Written by: Andrew Huebne')[1]
    
    BaSingSeScript = Toph_Katara + Iroh + Aang + Sokka + Zuko
    
    #
    bold_split = BaSingSeScript.split('<b>')
    
    #creates a list of lists with both character + relevant dialogue
    master_list = []
    for x in range(len(bold_split)):
        output = bold_split[x].split(':')
        master_list.append(output)
    
    #collects all the characters who speak
    character_list = []

    for x in range(len(master_list)):
        holder = master_list[x]
        if len(holder)>1:
            test = holder[0].strip()
            character_list.append(test.strip('</b>')) # remove the bold hyperlink to cleanup the data
    #collects what all the characters say
    dialogue_list = []

    for x in range(len(master_list)):
        holder = master_list[x]
        if len(holder)>1:
            test = holder[0].strip()
            dialogue_list.append(holder[1].strip('</b>')) # remove the bold hyperlink to cleanup the data 
    
    episode_list = []
    
    for x in range(len(character_list)):
       episode_list.append(215)
    
    dfScript = pd.DataFrame() 
    
    dfScript['episode_code'] = episode_list
    
    dfScript['character_list'] = character_list
    dfScript['dialogue_list'] = dialogue_list
    
    count = dfScript['dialogue_list'].str.split().str.len()
    count = list(count)
    dfScript['word_count']=count
    
    
    return(dfScript)

In [ ]:
#scriptCleanTalesofBaSingSe(RawScriptPull(215))

In [ ]:
def dfConsolidateScript(dfScript):

    #dfScript = scriptClean(x)
    
    i = 1

    dfScriptFinal = pd.DataFrame()
    dfScriptFinal = dfScriptFinal.append(dfScript.iloc[0]) #return another function

    while i < len(dfScript.index):
        if dfScript['character_list'][i] == dfScript['character_list'][i-1]: 

            #consolidates the word count into a single value when the same characters speaks twice in a row
            word_count_consolidate = (dfScript['word_count'][i]+dfScript['word_count'][i-1])
            dfScriptFinal['word_count'].at[i-1] = word_count_consolidate 

            #consolidates the dialogue when the same character speaks twice in a row
            dialogue_consolidate = (dfScript['dialogue_list'][i]+ " " + dfScript['dialogue_list'][i-1])
            dfScriptFinal['dialogue_list'].at[i-1] = dialogue_consolidate 

            i = i + 1
        else:
            dfScriptFinal = dfScriptFinal.append(dfScript.iloc[i])
            i = i + 1
    dfScriptFinal['episode_code'] = dfScriptFinal['episode_code'].astype(int)
    return(dfScriptFinal)

In [ ]:
#example of passing a functin to another function. Working better? Looks like it!
#dfConsolidateScript(scriptClean(RawScriptPull(215)))

In [ ]:
season_episode_list.remove('215') #removes Tales of Ba Sing Se

In [ ]:
# can't get list comprehension to work 
#string vs. int is why list comp wasn't working 

#season_episode_list = [e for e in season_episode_list if e not in to_remove]

In [ ]:
#to_skip = (212,215) #(understand what's going on with 212 & 215)

#trimmed_episode_list = []
#for i in season_episode_list:
 #   if int(i) not in to_remove: #season_episode_list is currently strings. Switched to int
  #      trimmed_episode_list.append(i)

In [ ]:
df_list_holder = []

for i in season_episode_list:
    #print(i)
    df_episode = dfConsolidateScript(scriptClean(RawScriptPull(i)))
    df_list_holder.append(df_episode)

df_master = pd.concat(df_list_holder)
df_master.reset_index(drop = True, inplace = True)

In [ ]:
df_master.tail()

In [ ]:
df_215 = dfConsolidateScript(scriptCleanTalesofBaSingSe(RawScriptPull(215)))

In [ ]:
df_215

In [ ]:
#combines episodes 215
df_final = df_master.append(df_215)

In [ ]:
df_final

In [ ]:
df_final.columns

In [ ]:
#Give the index a name
df_final.index.name = 'MyIndex'

In [ ]:
df_final.sort_values("episode_code")

In [ ]:
df_final.sort_values(['episode_code','MyIndex'],ascending = True,inplace = True)

#info on how to sorty by both column and index
#https://stackoverflow.com/questions/20277358/sort-pandas-dataframe-both-on-values-of-a-column-and-index

In [ ]:
df_final[df_final.episode_code == 215] #.sort_values(by = 'MyIndex')

In [ ]:
#reset index for a final time

In [ ]:
df_final.reset_index(drop = True, inplace = True)

In [ ]:
df_final[df_final['episode_code'] == 215]

In [ ]:
df_final.to_csv('ATLA_full_script.csv')

In [ ]:
#Next Steps... 
#where I need to clearn things ... 

# <u
# </b>

#dashes
# other special charcter?s